In [1]:
# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd

plt.ion()   # interactive mode

In [2]:
import pickle

In [3]:
train_imgs = pickle.load(open("train_images_512.pk",'rb'), encoding='bytes')
train_labels = pickle.load(open("train_labels_512.pk",'rb'), encoding='bytes')
test_imgs = pickle.load(open("test_images_512.pk",'rb'), encoding='bytes')

In [4]:
np.array(train_imgs).shape

X_messy = np.array(train_imgs)
X = np.zeros((70,512*512))
for i in range(70):
    for j in range(512):
        for k in range(512):
            X[i,j*512+k] = X_messy[i,1,j,k]
print(X.shape)
        
y = np.array(train_labels)

(70, 262144)


In [23]:
toShuffle = np.concatenate((X,y[:,np.newaxis]), axis=1)
np.random.shuffle(toShuffle)
shuffled_y = toShuffle[:,-1] 
shuffled_X = toShuffle[:, :-1]
print(shuffled_X)

[[-0.99996924 -0.99996924 -0.99996924 ... -0.99996924 -0.99996924
  -0.99996924]
 [-0.99310267 -0.99310267 -0.99305654 ... -0.99467129 -0.99469435
  -0.99471742]
 [-0.99738562 -0.99738562 -0.99738562 ... -0.99415612 -0.99427915
  -0.99437141]
 ...
 [-0.99801612 -0.99801612 -0.99801612 ... -0.99628603 -0.99635524
  -0.9964475 ]
 [-0.9928335  -0.9928335  -0.9928335  ... -0.99566323 -0.99569398
  -0.99572474]
 [-0.99523258 -0.9992618  -0.99932331 ... -0.99932331 -0.99932331
  -0.99861592]]


In [31]:
from sklearn.ensemble import RandomForestClassifier

X = shuffled_X
y = shuffled_y

a = np.arange(70)
for maxDepth in range(1, 5):
    tp = 0
    fn = 0
    fp = 0
    clf = convolutionForests(maxDepth = maxDepth)
    #clf = RandomForestClassifier(max_depth = maxDepth, random_state=0)
    for holdout in range(70):
        clf.fit(X[a%70 != holdout, :], y[a%70 != holdout])
        pred = clf.predict(X[np.newaxis, holdout, :])
        #print(str(pred[0]) + ","+str(y[holdout]))
        if (pred[0] == 1 and y[holdout] == 1):
            tp += 1 
        else:
            if((pred[0] == 1) and (y[holdout] == 0)):
                fp += 1
            else:
                if((pred[0] == 0) and (y[holdout] == 1)):
                    fn += 1
    p = tp/(tp+fn)
    r = tp/(tp+fp)
    f1 = 2*p*r/(p+r)
    print("MaxDepth: "+ str(maxDepth) + ", errorrate = "+ str((fp+fn)/70) + ", f1 = " + str(f1) + ", p = " + str(p) + ", r = " + str(r))

C:\Users\Hayde\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


MaxDepth: 1, errorrate = 0.22857142857142856, f1 = 0.864406779661017, p = 0.9272727272727272, r = 0.8095238095238095


C:\Users\Hayde\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


MaxDepth: 2, errorrate = 0.21428571428571427, f1 = 0.8672566371681416, p = 0.8909090909090909, r = 0.8448275862068966


C:\Users\Hayde\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


MaxDepth: 3, errorrate = 0.18571428571428572, f1 = 0.8807339449541284, p = 0.8727272727272727, r = 0.8888888888888888


C:\Users\Hayde\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


MaxDepth: 4, errorrate = 0.21428571428571427, f1 = 0.8598130841121494, p = 0.8363636363636363, r = 0.8846153846153846


In [29]:
from sklearn.ensemble import RandomForestClassifier

class convolutionForests:
    
    def __init__(self, maxDepth = 3):
        self.model = RandomForestClassifier(max_depth = maxDepth, random_state=0)
    def fit(self,X,y):
        self.convolve(X,y)
        self.model.fit(X,y)
    
    def convolve(self,X,y):
        #shifts:
        for shift in np.arange(1,4):
            Xshifted = np.zeros(X.shape)
            for row in range(X.shape[0]):
                Xshifted[row, shift:X.shape[1]] = X[row, 0:-1*shift]
            X = np.concatenate((Xshifted, X), axis = 0)
            y = np.concatenate((y,y), axis=0)
        #print(y)
    def predict(self, Xtest):
        return self.model.predict(Xtest)
        